In [1]:
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

from unet import UNet
from carvana_dataset import CarvanaDataset

if __name__ == "__main__":
    LEARNING_RATE = 3e-4
    BATCH_SIZE = 32
    EPOCHS = 5
    DATA_PATH = r"D:\\DATA\\CNNs\\Data\\UNET\\Wildfire\\Data"
    MODEL_SAVE_PATH = r"D:\\DATA\\CNNs\\Data\\UNET\\models\\unet_wildfire5epoch.pth"
   

    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_dataset = CarvanaDataset(DATA_PATH)

    generator = torch.Generator().manual_seed(42)
    train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2], generator=generator)

    train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=BATCH_SIZE,
                                shuffle=True)
    val_dataloader = DataLoader(dataset=val_dataset,
                                batch_size=BATCH_SIZE,
                                shuffle=True)

    model = UNet(in_channels=3, num_classes=1).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in tqdm(range(EPOCHS)):
        model.train()
        train_running_loss = 0
        for idx, img_mask in enumerate(tqdm(train_dataloader)):
            img = img_mask[0].float().to(device)
            mask = img_mask[1].float().to(device)

            y_pred = model(img)
            optimizer.zero_grad()

            loss = criterion(y_pred, mask)
            train_running_loss += loss.item()
            
            loss.backward()
            optimizer.step()

        train_loss = train_running_loss / (idx + 1)

        model.eval()
        val_running_loss = 0
        with torch.no_grad():
            for idx, img_mask in enumerate(tqdm(val_dataloader)):
                img = img_mask[0].float().to(device)
                mask = img_mask[1].float().to(device)
                
                y_pred = model(img)
                loss = criterion(y_pred, mask)

                val_running_loss += loss.item()

            val_loss = val_running_loss / (idx + 1)

        print("-"*30)
        print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
        print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
        print("-"*30)

    torch.save(model.state_dict(), MODEL_SAVE_PATH)


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [20:14<00:00, 43.37s/it]

 20%|████████████████                                                                | 1/5 [22:05<1:28:20, 1325.10s/it]

------------------------------
Train Loss EPOCH 1: 0.4542
Valid Loss EPOCH 1: 0.3225
------------------------------



100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [19:11<00:00, 41.11s/it]

 40%|████████████████████████████████                                                | 2/5 [42:40<1:03:37, 1272.39s/it]

------------------------------
Train Loss EPOCH 2: 0.2772
Valid Loss EPOCH 2: 0.2582
------------------------------



100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [20:23<00:00, 43.70s/it]

 60%|████████████████████████████████████████████████                                | 3/5 [1:04:05<42:36, 1278.20s/it]

------------------------------
Train Loss EPOCH 3: 0.2590
Valid Loss EPOCH 3: 0.2519
------------------------------



100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [14:09<00:00, 30.33s/it]

 80%|████████████████████████████████████████████████████████████████                | 4/5 [1:19:16<18:53, 1133.06s/it]

------------------------------
Train Loss EPOCH 4: 0.2540
Valid Loss EPOCH 4: 0.2533
------------------------------



100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [15:28<00:00, 33.15s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [1:35:46<00:00, 1149.22s/it]

------------------------------
Train Loss EPOCH 5: 0.2519
Valid Loss EPOCH 5: 0.2953
------------------------------


In [2]:
torch.cuda.empty_cache()